Background
----------

### Adaptive video

In general high-quality video requires a higher data rate than a
lower-quality equivalent. Consider the following two video frames. The
first shows a video encoded at 200kbps:

![](https://witestlab.poly.edu/blog/content/images/2016/02/dash-200.png)

Here’s the same frame at 500kbps, with noticeably better quality:

![](https://witestlab.poly.edu/blog/content/images/2016/02/dash-500.png)

For web services that want to share video with their users, this poses a
dilemma - what quality level should they use to encode the video? If a
video is low quality, it will stream without interruption even on a slow
3G cellular connection, but a user on a high speed fiber network may be
unhappy with the video quality. Or, the video may be high quality, but
then the slow connection would not be able to stream it without constant
interruptions.

Fortunately, there is a solution to this dilemma: adaptive video.
Instead of delivering exactly the same video to every user, adaptive
video delivers video that is matched to the individual user’s network
quality.

There are many different adaptive video products: Microsoft Smooth
Streaming, Apple HTTP Live Streaming (HLS), Adobe HTTP Dynamic Streaming
(HDS), and Dynamic Adaptive Streaming over HTTP (DASH). This experiment
focuses on DASH, which is widely supported as an international standard.

To prepare a video for adaptive video streaming with DASH, the video
file is first encoded into different versions, each having a different
rate and/or resolution. These are called *representations* or media
presentations. The representations of a video all have the same content,
but they differ in quality.

Each of these is further subdivided in time into *segments* of equal
lengths (e.g., four seconds).

![](https://witestlab.poly.edu/blog/content/images/2016/02/dash-stored.png)

The content server then stores all of the segments of all of the
representations (as separate files). Alongside these files, the content
server stores a manifest file, called the Media Presentation Description
(MPD). This is an XML file that identifies the various representations,
identifies the video resolution and playback rate for each, and gives
the location of every segment in each representation.

With these preparations complete, a user can begin to stream adaptive
video from the server!

Once the MPD and video files are in place, users can start requesting
DASH video.

First, the user requests the MPD file. It parses the MPD file, learns
what representations are available, and decides what representation to
request for the first segment. It then retrieves that specific file
using the URL given in the MPD.

The user’s device keeps a video buffer (at the application layer). As
new segments are retrieved, they are placed in the buffer. As video is
played back, it is removed from the buffer.

Each time a client finishes retrieving a file into the buffer, it makes
a new decision as to what representation to get for the next segment.

For example, the client might request the following representations for
the first four segments of video:

![](https://witestlab.poly.edu/blog/content/images/2016/02/dash-requested.png)

The cumulative set of decisions made by the client is called a decision
policy. The decision policy is a set of rules that determine which
representation to request, based on some kind of client state - for
example, what the current download rate is, or how much video is
currently stored in the buffer.

The decision policy is not specified in the DASH standard. Many decision
policies have been proposed by researchers, each promising to deliver
better quality than the next!

### DASH decision policies

The obvious policy to maximize video quality alone would be to always
retrive segments at the highest quality level. However, with this policy
the user is likely to experience rebuffering - when playback is
interrupted and the user has to wait for more video to be downloaded.
This occurs when the video is being played back (and therefore, removed
from the buffer) faster than it is being retrieved - i.e., the playback
rate is higher than the download rate - so the buffer becomes empty.
This state, which is known as buffer starvation, is obviously something
we wish very much to avoid.

To create a positive user experience for streaming video, therefore,
requires a delicate balancing act.

-   On the one hand, increasing the video playback rate too much (so
    that it is higher than the download rate) causes the undesired
    rebuffers.
-   On the other hand, decreasing the video playback rate also decreases
    the user-perceived video quality.

Performing rate selection to balance rebuffer avoidance and quality
optimization is an ongoing tradeoff. Different DASH policies may make
different decisions about how to balance that tradeoff. Different DASH
policies may also decide to use different pieces of information for
decision making. For example:

-   A decision policy may decide to focus on download rate in its
    decision making - select the quality level for the next video
    segment according to the download rate from the previous segment(s).
-   Or, a decision policy may focus on buffer occupancy (how much video
    is already downloaded into the buffer, waiting to be played back?)
    If there is already a lot of video in the buffer, the decision
    policy can afford to be aggressive in its quality selection, since
    it has a cushion to protect it from rebuffering. On the other hand,
    if there is not much video in the buffer, the decision policy should
    be careful not to select a quality level that is too optimistic,
    since it is at high risk of rebuffering.

### Specific policies in this implementation

In this experiment, we will use the DASH implementation developed for
the following paper:

> P. Juluri, V. Tamarapalli and D. Medhi, “SARA: Segment aware rate
> adaptation algorithm for dynamic adaptive streaming over HTTP,” 2015
> IEEE International Conference on Communication Workshop (ICCW), 2015,
> pp. 1765-1770, doi: 10.1109/ICCW.2015.7247436.

which is available on [Github](https://github.com/pari685/AStream). It
includes three DASH decision policies:

The “basic” policy selects the video rate that is one level lower than
the current network data rate. You can see [the “basic” implementation
here](https://github.com/pari685/AStream/blob/master/dist/client/adaptation/basic_dash2.py).

The buffer-based rate adaptation (“Netflix”) algorithm uses the
estimated network data rate only during the initial startup phase.
Otherwise, it makes quality decisions based on the buffer occupancy. It
is based on the algorithm described in the following paper:

> Te-Yuan Huang, Ramesh Johari, Nick McKeown, Matthew Trunnell, and Mark
> Watson. 2014. A buffer-based approach to rate adaptation: evidence
> from a large video streaming service. In Proceedings of the 2014 ACM
> conference on SIGCOMM (SIGCOMM ’14). Association for Computing
> Machinery, New York, NY, USA, 187–198.
> DOI:https://doi.org/10.1145/2619239.2626296

You can see [the “Netflix” implementation
here](AStream/dist/client/adaptation/netflix_dash.py).

Finally, the segment-aware rate adaptation (“SARA”) algorithm uses the
actual size of the segment and data rate of the network to estimate the
time it would take to download the next segment. Then, given the current
buffer occupancy, it selects the best possible video quality while
avoiding buffer starvation. It is described in

> P. Juluri, V. Tamarapalli and D. Medhi, “SARA: Segment aware rate
> adaptation algorithm for dynamic adaptive streaming over HTTP,” 2015
> IEEE International Conference on Communication Workshop (ICCW), 2015,
> pp. 1765-1770, doi: 10.1109/ICCW.2015.7247436.

You can see [the “SARA” implementation
here](AStream/dist/client/adaptation/weighted_dash.py).

This experiment explores the tradeoff between different metrics of video
quality (average rate, interruptions, and variability of rate) in an
adaptive video delivery system.

It should take about 60-120 minutes to run this experiment.

-   [Background](background.ipynb)
-   [Reserve resources on FABRIC](reserve_resources_fabric.ipynb)